In [104]:
import csgo_sims
import pandas as pd
import datetime
from random import uniform

In [105]:
team_stats = csgo_sims.TeamData("data/team_stats.csv")
matches = csgo_sims.Matches("data/matches.csv",types={"winner":str,"loser":str,"winnerscore":int,"loserscore":int,"date":"date","bo3":bool})
team_stats.get_elo(matches,"winner","loser","winnerscore","loserscore")
seeds = csgo_sims.SeedData("data/team_seeds.csv")
table = csgo_sims.Table(team_stats,seeds)

In [106]:
def get_tier(teamname,low = -6,high=11.00405885982721):
    assert teamname in team_stats.teams
    history = matches.find(teamname)
    tier = 0
    for idx in range(len(history)):
        winner = history.iloc[idx]["winner"]
        loser = history.iloc[idx]["loser"]
        if winner == teamname:
            if loser in team_stats.teams:
                ranking = team_stats[loser].stats["worldranking"]
                tier += (257/(ranking+256))**(25)
        else:
            if winner in team_stats.teams:
                ranking = team_stats[winner].stats["worldranking"]
                tier -= (ranking/257)**(1/3)
    return (tier - low)/high

In [107]:
def norm_form(teamname,
            sig_winstreak=5,winstreak_weight=4,last3monthsweight=1):
    #recent form
    team = team_stats.teams[teamname]
    tier = get_tier(teamname)
    winstreak = team.stats["winstreak"]
    try:
        winstreak = float(winstreak)
    except:
        winstreak = 0
    last3months = team.stats["win%last3months"]
    try:
        last3months = float(last3months)
    except:
        last3months = 0
    recent_form = (winstreak_weight*(float(winstreak)/sig_winstreak) + last3monthsweight * float())/100
    normalized_form = recent_form*tier
    return normalized_form

In [108]:
def on_paper(team1name,team2name,hltv_weight=0.8,seed_weight=0.2):
    team1 = team_stats.teams[team1name]
    team2 = team_stats.teams[team2name]
    total_hltv = team1.stats["hltv"] + team2.stats["hltv"]
    team1_hltv = team1.stats["hltv"]/total_hltv
    team2_hltv = team2.stats["hltv"]/total_hltv

    total_seed = team1.seed + team2.seed
    team1_seed = 1-(team1.seed/total_seed)
    team2_seed = 1-(team2.seed/total_seed)
    team1_total = (team1_seed * seed_weight + team1_hltv * hltv_weight)/(hltv_weight + seed_weight)
    team2_total = (team2_seed * seed_weight + team2_hltv * hltv_weight)/(hltv_weight + seed_weight)
    return team1_total/(team1_total + team2_total)

In [109]:
def get_history(team1name,team2name):
    wins = 0
    losses = 0
    team1 = team_stats.teams[team1name]
    team2 = team_stats.teams[team2name]
    history = matches.find(team1name)
    for idx in range(len(history)):
        match = history.iloc[idx]
        if match["winner"] == team1.name and match["loser"] == team2.name:
            wins += 1
        if match["loser"] == team1.name and match["winner"] == team2.name:
            losses += 1
    if (wins + losses) == 0:
        return 0.5
    else:
        return wins/(wins+losses)

In [110]:
def variabililty(teamname,worldrankingmax=53,weekstop30max=211,major_effect=2,optimal_age=26):
    team = team_stats.teams[teamname]
    rank = (worldrankingmax - team.stats["worldranking"])/worldrankingmax
    weekstop30 = (team.stats["weekstop30"]+major_effect)/(weekstop30max+major_effect)
    age = 1-(abs(team.stats["averageplayerage"]-optimal_age)/optimal_age)
    return (1 - (rank*weekstop30*age))**2

In [111]:
def rate(team1,team2,history_weight=0.1,form_weight=3,var_weight=1,upset_weight=0.2):
    team1rate = on_paper(team1,team2)
    team2rate = 1-team1rate
    historyteam1 = get_history(team1,team2)
    historyteam2 = 1-historyteam1
    team1rate += historyteam1*history_weight
    team2rate += historyteam2*history_weight
    total = team1rate + team2rate
    team1rate /= total
    team2rate /= total
    team1form = norm_form(team1)
    team2form = norm_form(team2)
    team1rate += team1form * form_weight
    team2rate += team2form * form_weight
    total = team1rate + team2rate
    team1rate /= total
    team2rate /= total
    team1var = variabililty(team1)
    temp = team1var
    team2var = variabililty(team2)
    team1var = team1var/team2var
    team2var = team2var/temp
    total = team1var + team2var
    team1var = team1var/total
    team2var = team2var/total
    if team1rate > team2rate:
        bad_game_chance = team1var
        good_game_chance = 1-team2var
        upset_chance = bad_game_chance*good_game_chance
        team2rate += upset_chance * upset_weight
    elif team1rate < team2rate:
        bad_game_chance = team2var
        good_game_chance = 1-team1var
        upset_chance = bad_game_chance*good_game_chance
        team1rate += upset_chance * upset_weight
    total = team1rate + team2rate
    team1rate /= total
    team2rate /= total
    return team1rate,team2rate

In [112]:
def simfunc(team1,team2):
    team1win,team2win = rate(team1.name,team2.name)
    bo3 = csgo_sims.isbo3(team1,team2)
    games_to_win = 1
    if bo3:
        games_to_win = 2
    team1games = 0
    team2games = 0
    team1totalrounds = 0
    team2totalrounds = 0
    while team1games != games_to_win and team2games != games_to_win:
        team1rounds = 0
        team2rounds = 0
        while team1rounds != 16 and team2rounds != 16:
            if uniform(0,1) < team1win:
                team1rounds += 1
            else:
                team2rounds += 1
        team1totalrounds += team1rounds
        team2totalrounds += team2rounds
        if team1rounds > team2rounds:
            team1games += 1
        else:
            team2games += 1
    if team1games > team2games:
        return team1,csgo_sims.GameStats(team1totalrounds-team2totalrounds,(team1games,team2games))
    else:
        return team2,csgo_sims.GameStats(team2totalrounds-team1totalrounds,(team2games,team1games))

In [114]:
table.do_sims(simfunc,nsims=1000)

Team               Promotion% (3-0)% (3-1)% (3-2)% (2-3)% (1-3)% (0-3)%
FaZe                     100%    60%    27%    13%     0%     0%     0%
Natus Vincere             99%    50%    43%     6%     1%     0%     0%
Heroic                    97%    27%    40%    30%     3%     0%     0%
Cloud9                    90%    12%    37%    41%     8%     2%     0%
FURIA                     86%    22%    33%    31%    11%     3%     0%
G2                        80%    11%    35%    34%    14%     6%     0%
ENCE                      75%     5%    28%    42%    20%     5%     0%
Outsiders                 61%     5%    27%    29%    27%    12%     0%
NIP                       41%     0%    12%    29%    31%    28%     0%
BIG                       24%     3%     6%    15%    46%    30%     0%
Vitality                  19%     0%     7%    12%    49%    31%     1%
Copenhagen Flames         14%     1%     4%     9%    59%    27%     0%
Liquid                    14%     4%     1%     9%    31%    52%

{'Heroic': {'promotion%': 0.97,
  '(3-0)%': 0.27,
  '(3-1)%': 0.4,
  '(3-2)%': 0.3,
  '(2-3)%': 0.03,
  '(1-3)%': 0.0,
  '(0-3)%': 0.0},
 'Copenhagen Flames': {'promotion%': 0.14,
  '(3-0)%': 0.01,
  '(3-1)%': 0.04,
  '(3-2)%': 0.09,
  '(2-3)%': 0.59,
  '(1-3)%': 0.27,
  '(0-3)%': 0.0},
 'BIG': {'promotion%': 0.24,
  '(3-0)%': 0.03,
  '(3-1)%': 0.06,
  '(3-2)%': 0.15,
  '(2-3)%': 0.46,
  '(1-3)%': 0.3,
  '(0-3)%': 0.0},
 'Cloud9': {'promotion%': 0.9,
  '(3-0)%': 0.12,
  '(3-1)%': 0.37,
  '(3-2)%': 0.41,
  '(2-3)%': 0.08,
  '(1-3)%': 0.02,
  '(0-3)%': 0.0},
 'FURIA': {'promotion%': 0.86,
  '(3-0)%': 0.22,
  '(3-1)%': 0.33,
  '(3-2)%': 0.31,
  '(2-3)%': 0.11,
  '(1-3)%': 0.03,
  '(0-3)%': 0.0},
 'FaZe': {'promotion%': 1.0,
  '(3-0)%': 0.6,
  '(3-1)%': 0.27,
  '(3-2)%': 0.13,
  '(2-3)%': 0.0,
  '(1-3)%': 0.0,
  '(0-3)%': 0.0},
 'NIP': {'promotion%': 0.41,
  '(3-0)%': 0.0,
  '(3-1)%': 0.12,
  '(3-2)%': 0.29,
  '(2-3)%': 0.31,
  '(1-3)%': 0.28,
  '(0-3)%': 0.0},
 'Natus Vincere': {'promotio